In [1]:
import os
import math
import shutil
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Define the directories where the data is stored for training, validation, and testing
train_dir = '/kaggle/input/gold-coin-dataset/train_data_directory/'
val_dir = '/kaggle/input/gold-coin-dataset/validation_data_directory/'
test_dir = '/kaggle/input/gold-coin-dataset/test_data_directory'

# Define the batch size and number of epochs for training
batch_size = 16
epochs = 10

# Define image dimensions
img_width, img_height = 150, 150

In [3]:
# Get the class names from the subdirectories of the training directory
class_names = sorted(os.listdir(train_dir))
class_names

['rsbl_100_995',
 'rsbl_100_999',
 'rsbl_10_995',
 'rsbl_10_999',
 'rsbl_1_995',
 'rsbl_1_999',
 'rsbl_20_995',
 'rsbl_20_999',
 'rsbl_2_995',
 'rsbl_2_999',
 'rsbl_50_995',
 'rsbl_50_999',
 'rsbl_5_995',
 'rsbl_5_999',
 'rsbl_8_995',
 'rsbl_8_999']

In [4]:
# Set up the data generator for training images with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_datagen

In [5]:
# Set up the data generator for validation and testing images (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 2585 images belonging to 16 classes.


In [7]:
# Load validation data from the directory with images
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  
)

Found 323 images belonging to 16 classes.


In [8]:
# Load testing data from the directory with images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 316 images belonging to 16 classes.


In [9]:
strategy = tf.distribute.MirroredStrategy()

In [10]:
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(class_names), activation='softmax')
    ])
    return model

In [11]:
# Create the model within the strategy scope
with strategy.scope():
    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
import tensorflow as tf
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Number of GPUs Available:  2


In [13]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
81/81 [==============================] - 261s 3s/step - loss: 1.4890 - accuracy: 0.5308 - val_loss: 0.7412 - val_accuracy: 0.6811
Epoch 2/10
81/81 [==============================] - 182s 2s/step - loss: 0.5362 - accuracy: 0.8112 - val_loss: 0.2385 - val_accuracy: 0.9257
Epoch 3/10
81/81 [==============================] - 181s 2s/step - loss: 0.3101 - accuracy: 0.8959 - val_loss: 0.1736 - val_accuracy: 0.9443
Epoch 4/10
81/81 [==============================] - 181s 2s/step - loss: 0.2269 - accuracy: 0.9188 - val_loss: 0.4553 - val_accuracy: 0.8050
Epoch 5/10
81/81 [==============================] - 207s 3s/step - loss: 0.1831 - accuracy: 0.9346 - val_loss: 0.0977 - val_accuracy: 0.9598
Epoch 6/10
81/81 [==============================] - 186s 2s/step - loss: 0.1384 - accuracy: 0.9563 - val_loss: 0.1112 - val_accuracy: 0.9598
Epoch 7/10
81/81 [==============================] - 174s 2s/step - loss: 0.1206 - accuracy: 0.9598 - val_loss: 0.1058 - val_accuracy: 0.9721
Epoch 8/10
81

In [14]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, steps=len(test_generator))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
# Get class names and their corresponding indices from the data generator
class_names = list(test_generator.class_indices.keys())
class_indices = test_generator.class_indices

# Print class names and their corresponding indices
print("Class Names and Indices:")
for class_name, class_index in class_indices.items():
    print(f"{class_name}: {class_index}")

10/10 [==============================] - 25s 2s/step - loss: 0.0429 - accuracy: 0.9747
Test Loss: 0.04288064315915108
Test Accuracy: 0.9746835231781006
Class Names and Indices:
rsbl_100_995: 0
rsbl_100_999: 1
rsbl_10_995: 2
rsbl_10_999: 3
rsbl_1_995: 4
rsbl_1_999: 5
rsbl_20_995: 6
rsbl_20_999: 7
rsbl_2_995: 8
rsbl_2_999: 9
rsbl_50_995: 10
rsbl_50_999: 11
rsbl_5_995: 12
rsbl_5_999: 13
rsbl_8_995: 14
rsbl_8_999: 15


In [15]:
model.save('image_classification_model.h5')

In [16]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

# Define the class names (replace with your class names)
class_names = sorted(os.listdir("/kaggle/input/gold-coin-dataset/train_data_directory/"))

# Load the trained model
model = load_model('image_classification_model.h5')

# Define the test directory
test_dir = "/kaggle/input/gold-coin-dataset2/test_data_directory/"

# Get the list of class subdirectories in the test directory
class_dirs = sorted(os.listdir(test_dir))

# Test two random images from each class
for class_dir in class_dirs:
    # Create the full path to the class subdirectory
    class_path = os.path.join(test_dir, class_dir)

    # Get the list of image file names in the class subdirectory
    image_files = os.listdir(class_path)

    # Check if there are at least two images in the class
    if len(image_files) < 2:
        print(f"Not enough images in class '{class_dir}'. Skipping testing.")
        continue

    # Select two random images from the class
    random_image_files = random.sample(image_files, 2)

    for image_file in random_image_files:
        # Create the full path to the test image
        test_image_path = os.path.join(class_path, image_file)

        # Load and preprocess the test image
        img = load_img(test_image_path, target_size=(150, 150))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0  # Rescale pixel values to [0, 1]

        # Make the prediction
        predictions = model.predict(img_array)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_class_index]

        print("Test Image:", test_image_path)
        print("Predicted Class:", predicted_class)
        print()

1/1 [==============================] - 0s 232ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/mZleE3miKa.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 21ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_995/IMG_20191114_142746.jpg
Predicted Class: rsbl_100_995

1/1 [==============================] - 0s 21ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/fc16I8WiZt.png
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 20ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_100_999/IMG_20191114_143455.jpg
Predicted Class: rsbl_100_999

1/1 [==============================] - 0s 20ms/step
Test Image: /kaggle/input/gold-coin-dataset2/test_data_directory/rsbl_10_995/_F0A5535 - Copy (6).png
Predicted Class: rsbl_8_995

1/1 [==============================] - 0s 20ms/step
Test Image: /kaggle/input/gold-coin-dataset